In [1]:
# Let's look at the FFTs of an image BEFORE and AFTER motion correction to see which yields the cleaner FFT.
# We already know the SA lattice doesn't move much, and any translation would result in a phase shift in the FFT
# without affecting the magnitudes of the spatial frequencies in the image.

In [2]:
# Import packages

from funcs_mrcio import iwrhdr_opened, irdhdr_opened, iwrsec_opened, irdpas_opened
from funcs_mrcio import irdsec_opened
import matplotlib.pyplot as plt
import numpy as np
import scipy.fft as sfft
import time
from skimage.filters import gaussian
from skimage.exposure import rescale_intensity
import pyfftw
import multiprocessing
import os
from tqdm import tqdm
import time
from PIL import Image

In [3]:
def rebin_helper(array, bin_size=2):
    """Take a two dimensional pixel array and average each bin_size x bin_size square to one pixel."""
    shape = (array.shape[0] // bin_size, bin_size,
             array.shape[1] // bin_size, bin_size)
    return array.reshape(shape).mean(-1).mean(1)

In [4]:
Image.MAX_IMAGE_PIXELS = None

In [5]:
os.chdir('/home/rkarimi/saxtal/motion_correction_average')

In [6]:
image = Image.open('SFO2_100_0001.tif')

In [7]:
num_frames = image.n_frames
print(num_frames)

50


In [8]:
# Create a loop to assemble a stack of rebinned images for a batch FFT

frames = []

for i in tqdm(range(num_frames)):
    image.seek(i)
    frame = np.transpose(rescale_intensity(rebin_helper(np.array(image), 2)))
    frames.append(frame)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:42<00:00,  1.17it/s]


In [9]:
big_array = np.stack(frames, axis=-1).astype('float32')

In [10]:
big_array.dtype

dtype('float32')

In [11]:
# Importing the library
import psutil

# Getting % usage of virtual_memory ( 3rd field)
print('RAM memory % used:', psutil.virtual_memory()[2])
# Getting usage of virtual_memory in GB ( 4th field)
print('RAM Used (GB):', psutil.virtual_memory()[3]/1000000000)

RAM memory % used: 56.8
RAM Used (GB): 18.529120256


In [12]:
del frames

In [13]:
np.save('SFO2_100_0001/frame_stack_float32.npy', big_array)